# Information
Title: Artificial Intelligence in Insurance Claims Management - Computer vision for car damage recognition 

Author: Roman Kastl

# Training & Fine-tuning EfficientNetB0 - Module 2 - vF
- Use transfer learning to classify images of damaged vs. non-damaged car
- EfficientNetB0


# Build, train, and save models

## Preparation

In [ ]:
!git clone https://github.com/djehuty94/MasterThesis_CarDamageRecognition
!pip install -U tensorflow-addons

### Import the required libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import os
import pathlib
import PIL
import random
import json
import datetime

#Only if we use colab
#from google.colab import files
from tensorflow.python.client import device_lib
import torch

#Import tensorboard plugins
import tensorboard as tb
from scipy import stats
from tensorboard.plugins.hparams import api as hp
from packaging import version
import seaborn as sns

#Library for model scheme
import pydot
import graphviz
import pydotplus

#Require tensorflow >= 2.3
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
import tensorflow_addons as tfa
print(tf.__version__)

In [ ]:
# Name variables for output file
model_version = "vF"
step_name ="Training and Fine-tuning"
module_name = "Module2"
save_path = ""

# Dummy variable for 
save_shape_plot = True
save_training_plot = True 
save_model = True
save_history = True

### Set seed value

In [ ]:
# Set the Random Seed
seed_value= 123

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

### Check that model will run on GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
torch.cuda.is_available()
device_lib.list_local_devices()

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

## Part 1 - Data Preprocessing


### - Recover data

From Google drive

In [ ]:
data_dir = pathlib.Path("MasterThesis_CarDamageRecognition/Data/Dataset_v2_cat_location")

### - Count the number of existing image and display

In [ ]:
#count the total number of image
image_count = len(list(data_dir.glob('*/*/')))
print(image_count)

### Pre-process the images and prepare the training, validation and test set
This part split the dataset in a training (80%) and test dataset (20%)

In [ ]:
#Preprocess the picture and prepare the set keras
batch_size = 32
img_size = (224,224)

In [ ]:
#Generate a training and validation dataset, in the next cell, a test dataset is generated from the validation dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='categorical',
  image_size= img_size ,
  shuffle=True,
  batch_size=batch_size)
  
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode='categorical',
  image_size=img_size,
  shuffle=True,
  batch_size=batch_size)

In [ ]:
#Skip the test set and only use a training and validation set for Module 2 and 3, due to the limited quantity of training data

#As not test dataset is available, we will extract a few pictures from the validation dataset
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_ds))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_ds))

In [ ]:
target_dict={k: v for v, k in enumerate(np.unique(train_ds.class_names))}
class_names = np.array(train_ds.class_names)
num_classes = len(class_names)
print(class_names)
print(num_classes)

### Display a few images of each class

As we can observed, 0 refers to the damage class and 1 to regular car pictures

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
#Print dataset format
# Here we use a batch of 32, the image dimensions are 224x224 and RGB (3)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Configure dataset for performance

In [ ]:
#Optimize the datasets for performances
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

## Part 2 - Build the model

### Build Callbacks and Training method

Callback: 

- EarlyStopping: Stop training the model if loss on validation set has not improved over 3 iterations
- ModelCheckpoint: Save the best model based on the validation accuracy
- WIP - TensorBoard: Generate the data for a tensorboard 


Model:
- EfficientNet-B0

In [ ]:
def train_model(model_to_train,callbacks, epochs):
    model_to_train
    history_model_to_train = model_to_train.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks
    )
    return(model_to_train,history_model_to_train)

In [ ]:
### COMPILE THE MODEL
if num_classes == 2:
    loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    METRICS = [
             'accuracy',
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'),             
             tfa.metrics.F1Score(name='f1-score',average='macro',num_classes=num_classes,threshold=0.5),
             ]
else:
    loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    METRICS = [
             'accuracy',
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'),             
             tfa.metrics.F1Score(name='f1-score',average='macro',num_classes=num_classes,threshold=0.5),
             tfa.metrics.F1Score(name='f1-score_perClass',num_classes=num_classes,threshold=0.5)             
             ]

In [ ]:
def build_callback(model_name, patience, verbose):
    callbacks_list = [
     keras.callbacks.EarlyStopping(
         monitor="val_loss",
         patience=patience,
         verbose=verbose,
         mode="auto",
         restore_best_weights=True,
     ),
      keras.callbacks.TensorBoard(
          log_dir="logs/fit/"+model_name,
          histogram_freq=1,
          embeddings_freq=1,
          )]
    if save_model == True:
      {
        callbacks_list.append(
         keras.callbacks.ModelCheckpoint(
         filepath=save_path+""+model_name+".h5",
         monitor="val_loss",
         save_best_only=True,
     ))
      }

    return(callbacks_list)


### Data augmentation

In this code data augmentation is achieved through layers which are added to the model

In [ ]:
#Create layer for image augmentation
#It will help to prevent overfitting
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.15),
    layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

### Build and Compile EfficientNetB0
- Initialize
- Pass Data augmentation
- Preprocess the input specially for EfficientNetB0
- Add top set of layer (GlobalMaxPooling2D + BatchNorm + Dropout 0.5)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

#Make the EfficientNetB0 model
def make_model_EfficientNetB0(input_shape, num_classes, model_arch_name):

    ### BUILD THE MODEL
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)

    preprocess_input_EfficientNetB0 = keras.applications.efficientnet.preprocess_input
    x = preprocess_input_EfficientNetB0(x)
    
    #We do not want the base to be trainable, otherwise we would lose all the advantagres of using pre-trained model
    #conv_base_EfficientNetB0  = EfficientNetB0(weights='imagenet',include_top=False,input_tensor=x, drop_connect_rate=0.2)      
    conv_base_EfficientNetB0  = EfficientNetB0(weights='imagenet',include_top=False,input_tensor=x)      
    conv_base_EfficientNetB0.trainable = False
    conv_base_EfficientNetB0.summary()

    #Rebuild top, starting with a GlobalMaxPooling in order to convert the feature maps in vectors
    x = keras.layers.GlobalMaxPooling2D(name="top_GlobalMaxPooling2D")(conv_base_EfficientNetB0.output)
    x = layers.BatchNormalization(name="topBatchNorm")(x)

    dropout_rate = 0.5
    x = layers.Dropout(dropout_rate, name="top_dropout")(x)    

    activation = "softmax"
    units = num_classes

    outputs = layers.Dense(units, activation=activation, name="pred")(x)
    model = keras.Model(inputs, outputs, name="EfficientNetB0")
    
    #a learning rate of 1e-2 reduced the valuation f1
    opt = Adam(lr=1e-3)
    
    model.compile(optimizer=opt, loss=loss_func, metrics=METRICS)
    
    #model.summary()
    if (save_shape_plot == True):
      keras.utils.plot_model(model,show_shapes=True,to_file=save_path+model_arch_name+"_plot.png")
    return(model)

### Build chart to evaluate the model training process

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_metrics(history, key):
  metrics =  ['accuracy','loss','f1-score']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history[metric], color=colors[0], label='Train')
    plt.plot(history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    else:
      plt.ylim([0,1])

    plt.legend()
    fig = plt.gcf()
    if(save_training_plot == True):
      fig.savefig(save_path+key+"_training_plot.png", dpi=300)

### JSON export

In [ ]:
def json_export(name, data):
  df = pd.DataFrame.from_dict(data)
  csv_path = save_path+module_name+"_"+model_version+"_history_df_"+name+".csv"
  df.to_csv(csv_path, index=False)

## Part 3 - Compile, train and assess EfficientNet-B0
Can only run one of the below, must restart the session in between the compilation of two model. The goal is to uniform the tests

###Compile and summary of EfficientNet-B0

In [ ]:
model_EfficientNetB0 = make_model_EfficientNetB0(img_size+(3,),num_classes, "EfficientNetB0")
model_EfficientNetB0.summary()

### Train EfficientNetB0

In [ ]:
#Set to 30 because we do not want early stopping for cause of Visual rendering, but we still use early stopping in order to restore the best weights
epochs = 30
patience = epochs
verbose = 1

callbacks_EfficientNetB0 = build_callback("EfficientNetB0", patience, verbose)
model_EfficientNetB0, history_model_EfficientNetB0 = train_model(model_EfficientNetB0, callbacks_EfficientNetB0, epochs)
# Save it under the form of a json file
if save_history == True:
  json_export("EfficientNetB0", history_model_EfficientNetB0.history)

### Chart the model EfficientNet-B0

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
plot_metrics(history_model_EfficientNetB0.history,"EfficientNetB0")

## Part 4 - Fine-tune the model

### Determine layers to unfreeze

In [ ]:
 #model_EfficientNetB0.summary()
for layer in model_EfficientNetB0.layers[-9:]:
  print(layer)

### Build unfreeze_model method

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers, corresponding to the Block 7 of the EfficientNet-B0 model, use 21 if the Dense 128 layer is added on top
    for layer in model.layers[-9:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    #Lower learning rate to 1e-4 for fine-tuning
    opt = Adam(lr=1e-4)
    
    model.compile(optimizer=opt, loss=loss_func, metrics=METRICS)

unfreeze_model(model_EfficientNetB0)

In [ ]:
model_EfficientNetB0.summary()

### Train the fine-tuned model

In [ ]:
#Set to 30 because we do not want early stopping for cause of Visual rendering, but we still use early stopping in order to restore the best weights
epochs = 30
patience = epochs
verbose = 1

callbacks_EfficientNetB0 = build_callback("EfficientNetB0_ft", patience, verbose)
model_EfficientNetB0, history_model_EfficientNetB0 = train_model(model_EfficientNetB0, callbacks_EfficientNetB0, epochs)
# Save it under the form of a json file
if save_history == True:
  json_export("EfficientNetB0_ft", history_model_EfficientNetB0.history)

### Chart the fine-tuned model

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
plot_metrics(history_model_EfficientNetB0.history,"EfficientNetB0_ft")

# Tensorboard

In [ ]:
#Update to Tensorboard
!tensorboard dev upload --logdir ./logs \
  --name "Training & Fine-tuning - Module 2 - vF" \
  --description "EfficientNet-B0 - Max Pooling, Batch Normalization, Dropout = 0.5" \
  --one_shot

## Access Tensorboard and build chart

https://www.tensorflow.org/tensorboard/dataframe_api

In [ ]:
experiment_id = "ID RETURNED ABOVE"

experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
dfw = experiment.get_scalars(pivot=True) 
dfw

In [ ]:
csv_path = save_path+module_name+"_"+model_version+"_experiment.csv"
dfw.to_csv(csv_path, index=False)
dfw_roundtrip = pd.read_csv(csv_path)
pd.testing.assert_frame_equal(dfw_roundtrip, dfw)

In [ ]:
dfw

In [ ]:
# Filter the DataFrame to only validation data, which is what the subsequent
# analyses and visualization will be focused on.
dfw_validation = dfw[dfw.run.str.endswith("/validation")]
# Get the optimizer value for each row of the validation DataFrame.
optimizer_validation = dfw_validation.run.apply(lambda run: run.split(",")[0])

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
sns.lineplot(data=dfw_validation, x="step", y="epoch_f1-score", 
             hue=optimizer_validation).set_title("f1-score")
plt.subplot(1, 2, 2)
sns.lineplot(data=dfw_validation, x="step", y="epoch_loss",
             hue=optimizer_validation).set_title("loss")

In [ ]:
dfw.groupby('run')['epoch_f1-score'].nlargest(1,)


### EfficientNet-B0

In [ ]:
dfw_EfficientNetB0 = dfw[dfw.run.str.startswith("fit/EfficientNetB0/")]

In [ ]:
# Get the optimizer value for each row of the validation DataFrame
optimizer_validation = dfw_EfficientNetB0.run.apply(lambda run: run.split(",")[0])

plt.figure(figsize=(20, 6))
plt.subplot(1, 3, 1)
sns.lineplot(data=dfw_EfficientNetB0, x="step", y="epoch_f1-score", 
             hue=optimizer_validation).set_title("f1-score")
plt.subplot(1, 3, 2)
sns.lineplot(data=dfw_EfficientNetB0, x="step", y="epoch_accuracy",
             hue=optimizer_validation).set_title("accuracy")
plt.subplot(1, 3, 3)
sns.lineplot(data=dfw_EfficientNetB0, x="step", y="epoch_loss",
             hue=optimizer_validation).set_title("loss")

In [ ]:
dfw_EfficientNetB0_ft = dfw[dfw.run.str.startswith("fit/EfficientNetB0_ft")]

In [ ]:
# Get the optimizer value for each row of the validation DataFrame.
optimizer_validation = dfw_EfficientNetB0_ft.run.apply(lambda run: run.split(",")[0])

plt.figure(figsize=(20, 6))
plt.subplot(1, 3, 1)
sns.lineplot(data=dfw_EfficientNetB0_ft, x="step", y="epoch_f1-score", 
             hue=optimizer_validation).set_title("f1-score")
plt.subplot(1, 3, 2)
sns.lineplot(data=dfw_EfficientNetB0_ft, x="step", y="epoch_accuracy",
             hue=optimizer_validation).set_title("accuracy")
plt.subplot(1, 3, 3)
sns.lineplot(data=dfw_EfficientNetB0_ft, x="step", y="epoch_loss",
             hue=optimizer_validation).set_title("loss")

# Load and evaluate models

## Part 1 - Load the model and evaluate its accuracy

In [ ]:
model = model_EfficientNetB0
model_name = "EfficientNetB0"

In [ ]:
def model_accuracy():
  loss, acc, precision, recall, f1, f1_perclass = model.evaluate(test_ds)
  print('Restored model '+model_name+', accuracy: {:5.2f}%'.format(100*acc))
  print(model.predict(test_ds).shape)

In [ ]:
model_accuracy()

## Part 2 - Proceed to multiple predictions


In [ ]:
def model_predict():
  #Retrieve a batch of images from the test set
  image_batch, label_batch = test_ds.as_numpy_iterator().next()
  predictions = model.predict_on_batch(image_batch)

  plt.figure(figsize=(10, 10))
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    x = class_names[np.argmax(label_batch[i])]
    y = class_names[np.argmax(predictions[i])]
    plt.title("Actual:"+ x +"\nPredicted:"+ y +"")
    plt.axis("off")
  return predictions

In [ ]:
predictions = model_predict()
#Retrieve a batch of images from the test set

In [ ]:
#Check
np.sum(predictions, axis=1)

## Part 3 - Single Prediction

In [ ]:
## Provide the prediction for a single image

#Choose image path
img_path = "IMG_PATH"

img = keras.preprocessing.image.load_img(
      img_path, target_size=img_size
  )

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

prediction = model.predict(img_array)

print(f"The algorithm says this image is:\n {prediction[0,0]:.2%} {class_names[0]}\n and {prediction[0,1]:.2%} {class_names[1]}\n and {prediction[0,2]:.2%} {class_names[2]}\n")
img